In [1]:
import pandas as pd 
import numpy as np 
import yaml
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [2]:
def load_config(cfg_path):
    with open(cfg_path, "r") as f:
        return yaml.safe_load(f)

In [3]:
cfg = load_config('./configs/jj.yaml')
train_data_path = cfg["paths"]["train_data"]
test_data_path = cfg["paths"]["test_data"]
submission_path = cfg["paths"]["submission"]
output_path = cfg["paths"]["output"]
model_name = cfg["model"]["model_name"]
model_path = cfg["model"]["model_path"]
embedding_model_name = cfg["model"]["embedding_model"]
prompt_template = cfg["prompt_template"]
batch_size = cfg["settings"]["batch_size"]

In [4]:
train = pd.read_csv(train_data_path)
test = pd.read_csv(test_data_path)
sample = pd.read_csv(submission_path)

In [5]:
grouped = train.groupby("인적사고")

In [6]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

100%|██████████| 23/23 [04:22<00:00, 11.40s/it]


In [7]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 1개
Range 0.1 - 0.2: 25개
Range 0.2 - 0.3: 191개
Range 0.3 - 0.4: 665개
Range 0.4 - 0.5: 1657개
Range 0.5 - 0.6: 3794개
Range 0.6 - 0.7: 6597개
Range 0.7 - 0.8: 7557개
Range 0.8 - 0.9: 2783개
Range 0.9 - 1.0: 112개


In [ ]:
res

{'감전': '작업자 교육 및 안전장구 착용 철저와 전기작업자 교육 및 피복관리 철저를 통한 재발 방지 대책.',
 '교통사고': '작업업 특별 안전 교육 실시, 신호수 교육 실시, 작업차량 후방 감지 센서 및 카메라 설치, 수시 위험성 평가 실시, 감독관 주체 특별 안전 교육 실시, 현장 점검 시 신호수 위치 점검 등으로 구성된 재발 방지 대책 및 향후 조치 계획.',
 '기타': '작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지 대책.',
 '깔림': '작업 개시 전 작업내용 숙지 및 안전교육 강화, 수시 현장 방문 및 점검을 통한 재발 방지 대책 강구.',
 '끼임': '작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독 철저에 대한 재발 방지 대책과 향후 조치 계획.',
 '넘어짐(기타)': '작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획.',
 '넘어짐(물체에 걸림)': '작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획.',
 '넘어짐(미끄러짐)': '이동통로 확보 관리와 작업 전 안전교육 철저 및 정기적 근로자 안전교육 시행을 통한 재발 방지 대책.',
 '떨어짐(10미터 이상)': '안전시설물 설치와 체계적인 안전교육 실시를 통한 재발 방지 대책 및 공사현장 작업중지명령과 안전교육 강화를 포함한 향후 조치 계획.',
 '떨어짐(2미터 미만)': '안전교육 실시와 현장 내 작업지시사항 철저 이행, 안전관리 교육 이행, 안전위험요소 제거 점검 및 대책 강구를 통한 재발 방지 대책.',
 '떨어짐(2미터 이상 ~ 3미터 미만)': '작업 전 안전교육 철저와 안전고리 이중결속 준수, 작업 중 안전시설물 확인 및 사고자 상태 수시 체크를 통한 재발 방지 대책 마련.',
 '떨어짐(3미터 이상 ~ 5미터 미만)': '작업 시 안전교육 및 보호구 착용과 안전매트 설치를 통한 재발 방지 대책

In [9]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [10]:
sample.head(3)

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774
1,TEST_001,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774
2,TEST_002,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774


In [11]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [12]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [13]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [16]:
grouped.head()

,ID,발생일시,사고인지 시간,날씨,기온,습도,공사종류,연면적,층 정보,인적사고,물적사고,공종,사고객체,작업프로세스,장소,부위,사고원인,재발방지대책 및 향후조치계획
0,TRAIN_00000,2023-12-31 오후 12:44,정규작업 -,맑음,1℃,30%,건축 / 건축물 / 근린생활시설,"4,892.77㎡","지상 14층, 지하 3층",떨어짐(5미터 이상 ~ 10미터 미만),없음,건축 > 철근콘크리트공사,건설자재 > 철근,설치작업,근린생활시설 / 내부,철근 / 고소,"고소작업 중 추락 위험이 있음에도 불구하고, 안전난간대, 안전고리 착용 등 안전장치...",고소작업 시 추락 위험이 있는 부위에 안전장비 설치.
1,TRAIN_00001,2023-12-30 오후 03:35,정규작업 -,맑음,10℃,90%,토목 / 터널 / 철도터널,-,-,끼임,없음,토목 > 터널공사,건설자재 > 볼트,운반작업,철도터널 / 내부,볼트 / 바닥,부주의,재발 방지 대책 마련과 안전교육 실시.
2,TRAIN_00002,2023-12-30 오후 02:30,정규작업 -,맑음,14℃,70%,건축 / 건축물 / 업무시설,"1,994.62㎡","지상 5층, 지하 0층",넘어짐(미끄러짐),없음,건축 > 철근콘크리트공사,기타 > 기타,형틀 및 목공,업무시설 / 내부,기타 / 바닥,3층 슬라브 작업시 이동중 미끄러짐,현장자재 정리와 안전관리 철저를 통한 재발 방지 대책 및 공문 발송을 통한 향후 조...
3,TRAIN_00003,2023-12-30 오후 12:00,휴일근무 -,흐림,12℃,55%,토목 / 하천 / 기타,-,-,기타,전도,토목 > 하천공사,부재 > 교각 기초,조립작업,기타 / 동산교 신축구간,교각 기초 / 바닥,"교각 기초철근 조립 중 강한 바람에 의해 기둥측 주철근이 균형을 잃고 전도되어, 하...","위험성 평가 및 교육을 통해 작업장 내 위험요인과 안전수칙을 근로자에게 전파하고, ..."
4,TRAIN_00004,2023-12-30 오전 10:00,정규작업 -,맑음,0℃,10%,건축 / 건축물 / 공동주택,"59,388.93㎡","지상 27층, 지하 3층",넘어짐(미끄러짐),없음,건축 > 해체 및 철거공사,기타 > 건설폐기물,해체작업,공동주택 / 내부,건설폐기물 / 바닥,근로자의 부주의,자재 정리 작업 시 세부 작업 방법에 대한 교육 실시와 작업 구간 이동 경로 점검 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3602,TRAIN_03602,2023-06-01 오전 08:45,정규작업 -,흐림,17℃,80%,건축 / 건축물 / 숙박시설,"83,480.45㎡","지상 2층, 지하 12층",질식,화재,건축 > 건축물 부대공사,건설공구 > 공구류,마감작업,숙박시설 / 내부,공구류 / 상부(위),국립과학수사연구원 조사(6.7일) 예정이나 현재 인테리어 용접 시 화재가 난것으로 추정,화재 예방 철저 등 행정 지시와 국과수 조사 결과에 따른 행정 조치 계획.
4413,TRAIN_04413,2023-04-20 오후 01:00,정규작업 -,맑음,22℃,40%,건축 / 건축물 / 공장,"1,408㎡","지상 3층, 지하 0층",떨어짐(분류불능),없음,건축 > 건축물 부대공사,건설자재 > 자재,설치작업,공장 / 내부,자재 / 상부(위),안전난간 설치 불가구간에 생명줄 설치하였으나 재해자 안전대의 안전고리 미체결로 인한...,안전고리 생명줄 체결과 산재처리 및 해당공종 근로자 특별안전교육 실시.
5025,TRAIN_05025,2023-03-18 오전 10:23,휴일근무 -,맑음,10℃,28%,건축 / 건축물 / 운수시설,"88,544.26㎡","지상 5층, 지하 1층",질식,없음,건축 > 철근콘크리트공사,시설물 > 건물,절단작업,운수시설 / 내부,건물 / e/v홀 내부,내부공간에서 엔진커팅기를 사용하여 커팅 작업 시 환기가 제대로 않았음.(배기 장치...,"안전교육을 통한 근로자 안전의식 강화, 내부공간에서의 엔진식 공구 사용 지양, 2인..."
5852,TRAIN_05852,2023-01-31 오후 05:40,정규작업 -,흐림,7℃,20%,건축 / 건축물 / 공동주택,"256,322.41㎡","지상 36층, 지하 2층",질식,없음,건축 > 철근콘크리트공사,기타 > 기타,양생작업,공동주택 / 내부,기타 / 바닥,야자숯탄 점화로 콘크리트 양생 중 추가 점검 시기 전에 양생 현장에 투입 후 사고 ...,현장 근로자 안전 교육 실시 및 사고 원인 조사 후 안전 조치 마련.
